# Concatenation with Validation

In [2]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from scipy.stats import gaussian_kde
from quantum_circuits import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
path1 = '../../../data/'
path2 = 'data/'

In [4]:
def fit_model(
    channel=None,
    spam=None,
    N_map=None,
    N_spam=None,
    loss_function=None,
    num_iter_pretrain=300,
    num_iter_map=2000,
    num_iter_spam=2000,
    filename=None,
    ratio=0.9,
    verbose=True,
    counts=False,
):
    if not counts:
        inputs_map, targets_map, inputs_spam, targets_spam = pickle.load(
            open(filename, "rb")
        )
    else:
        inputs_map, inputs_spam, counts = pickle.load(open(filename, "rb"))
        targets = counts_to_probs(counts)
        N = inputs_map[0].shape[0]
        targets_map = targets[:N]
        targets_spam = targets[N:]

    inputs_val_map = None
    targets_val_map = None    

    if ratio is not None:
        inputs_map, targets_map, inputs_val_map, targets_val_map = train_val_split(
            inputs_map, targets_map, ratio=ratio
        )

    if num_iter_pretrain != 0:
        spam.pretrain(num_iter=num_iter_pretrain, verbose=False)

    spam.train(
        inputs=inputs_spam,
        targets=targets_spam,
        num_iter=num_iter_spam,
        N=N_spam,
        verbose=verbose,
    )

    channel.spam = spam
    model = ModelQuantumMap(
        channel=channel,
        loss_function=loss_function,
        optimizer=tf.optimizers.Adam(learning_rate=0.01),
        logger=Logger(
            loss_function=KLDiv(),
            loss_function_val=KLDiv(),
            verbose=verbose,
        ),
    )

    model.train(
        inputs=inputs_map,
        targets=targets_map,
        inputs_val=inputs_val_map,
        targets_val=targets_val_map,
        num_iter=num_iter_map,
        N=N_map,
        verbose=verbose,
    )
    model.optimizer = None
    spam.optimizer = None
    model.inputs = None
    model.targets = None
    # model.inputs_val = None
    # model.targets_val = None

    return model

## Two Layer

In [6]:
n = 3
d = 2**n
model_list = []
verbose = False

np.random.seed(42)
random.seed(42)
for i in tqdm(range(10)):
    model1 =fit_model(channel=KrausMap(d=d, 
                                       rank=d**2),
                      spam=SPAM(init = InitialState(d),
                                povm = CorruptionMatrix(d),
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                ),
                      loss_function = ProbabilityMSE(),
                      N_map=500,
                      N_spam=None,
                      num_iter_pretrain=500,
                      num_iter_map=2000,
                      num_iter_spam=500,
                      filename= path1 + f"belem_concatenate_2layer{i}0",
                      verbose=verbose
                      )
    
    model2 =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename= path1 + f"belem_concatenate_2layer{i}1",
                  verbose=verbose
                  )

    model3 =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename=path1 + f"belem_concatenate_2layer{i}2",
                  verbose=verbose
                  )
    
    model_list.extend([model1, model2, model3])

saver(model_list, path2 + "belem_concatenate_2layer_fitted_validation.model")

  0%|          | 0/10 [00:00<?, ?it/s]

0.012880254023019266 0.014154086420546158
0.014110227124788082 0.0159638888864774
0.009923032654485905 0.0117649509074435
0.011062618234196993 0.012428395136744392
0.01411496241618938 0.01502655919919741
0.008410721238439896 0.011252007165153646
0.01016280996918193 0.012645087529424913
0.01079793200719572 0.012718801833191783
0.00931763288573401 0.01084257197532403
0.01148415941738357 0.014492220472866333
0.009339229031142699 0.011869958931561416
0.009314895287541867 0.011637354968385957
0.011570483700747595 0.014467075368303273
0.012095262524511497 0.01439478597745797
0.008618119246340919 0.012100368852485116
0.010872562770303176 0.01085517097675284
0.012112360139806792 0.013955890975064776
0.00931793367056932 0.010961395202497786
0.01300447351241134 0.016166179609214147
0.03724682326328094 0.017597003846929395
0.010595266876999474 0.013169269133175985
0.011731524479609004 0.014996746356885499
0.014375029685013824 0.01632989845519464
0.010576944032724061 0.01457998230318594
0.01155103

## Four Layer

In [6]:
n = 3
d = 2**n
model_list = []

verbose = False

np.random.seed(42)
random.seed(42)
for i in tqdm(range(10)):
    model1 =fit_model(channel=KrausMap(d=d, 
                                       rank=d**2),
                      spam=SPAM(init = InitialState(d),
                                povm = CorruptionMatrix(d),
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                ),
                      loss_function = ProbabilityMSE(),
                      N_map=500,
                      N_spam=None,
                      num_iter_pretrain=500,
                      num_iter_map=2000,
                      num_iter_spam=500,
                      filename= path1 + f"belem_concatenate_4layer{i}0",
                      verbose=verbose
                      )
    
    model2 =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename= path1 + f"belem_concatenate_4layer{i}1",
                  verbose=verbose
                  )

    model3 =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename=path1 + f"belem_concatenate_4layer{i}2",
                  verbose=verbose
                  )
    
    model_list.extend([model1, model2, model3])

saver(model_list, path2 + "belem_concatenate_4layer_fitted_validation.model")

  0%|          | 0/10 [00:00<?, ?it/s]

## Six Layers

In [ ]:
n = 3
d = 2**n
model_list = []
verbose = False

np.random.seed(42)
random.seed(42)
for i in tqdm(range(10)):
    model1 =fit_model(channel=KrausMap(d=d, 
                                       rank=d**2),
                      spam=SPAM(init = InitialState(d),
                                povm = CorruptionMatrix(d),
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                ),
                      loss_function = ProbabilityMSE(),
                      N_map=500,
                      N_spam=None,
                      num_iter_pretrain=500,
                      num_iter_map=2000,
                      num_iter_spam=500,
                      filename= path1 + f"belem_concatenate_6layer_retry{i}0",
                      verbose=verbose
                      )
    
    model2 =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename= path1 + f"belem_concatenate_6layer_retry{i}1",
                  verbose=verbose
                  )

    model3 =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename=path1 + f"belem_concatenate_6layer_retry{i}2",
                  verbose=verbose
                  )
    
    model_list.extend([model1, model2, model3])

saver(model_list, path2 + "belem_concatenate_6layer_fitted_validation.model")

  0%|          | 0/10 [00:00<?, ?it/s]

## Eight Layer

In [ ]:
n = 3
d = 2**n
model_list = []
verbose=False
np.random.seed(42)
random.seed(42)
for i in tqdm(range(10)):
    model1 =fit_model(channel=KrausMap(d=d, 
                                       rank=d**2),
                      spam=SPAM(init = InitialState(d),
                                povm = CorruptionMatrix(d),
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                ),
                      loss_function = ProbabilityMSE(),
                      N_map=500,
                      N_spam=None,
                      num_iter_pretrain=500,
                      num_iter_map=2000,
                      num_iter_spam=500,
                      filename= path1 + f"belem_concatenate_8layer{i}0",
                      verbose=verbose
                      )
    
    model2 =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename= path1 + f"belem_concatenate_8layer{i}1",
                  verbose=verbose
                  )

    model3 =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename=path1 + f"belem_concatenate_8layer{i}2",
                  verbose=verbose
                  )
    
    model_list.extend([model1, model2, model3])

saver(model_list, path2 + "belem_concatenate_8layer_fitted_validation.model")

  0%|          | 0/10 [00:00<?, ?it/s]

0.009559947015991492 0.9610987289096521
0.010645205387321917 0.959256943931556
0.070443142093342 0.7063998866810717
0.010026004848598918 0.959096446283811
0.010719861115193735 0.9605564625505422
0.007132341922483552 0.9515507776237461
0.009096888368549053 0.9601195542165304
0.00920100033898436 0.9637090020059166
0.007671958389386431 0.9455084877498297
0.009431951583558081 0.9643572349975069
0.009378547585894391 0.9612869085337533
0.007474024099643588 0.9490230026605977
0.008590070845308209 0.9622277347453535
0.00928101474151017 0.9639085174198805
0.007838890894485093 0.9340439045942821
0.010161555167365374 0.9597136044290223
0.01058715349977579 0.9605331802229485
0.007862511084456954 0.9434738927691035
0.010429349838781717 0.9525402347342716
0.04810534825025958 0.8355729120236783
0.007691380015584064 0.9429493536498735
0.009789634046802349 0.9590046784002915
0.009522279653597154 0.961770792599453
0.00769469300301615 0.9461364189307292
0.009512937541426035 0.9566437146555505
0.009916234

## Unitary Models

In [7]:
n = 3
d = 2**n
model_list = []
verbose = False

np.random.seed(42)
random.seed(42)
for i in tqdm(range(10)):

    model =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename=path1 + f"belem_concatenate_2layer{i}2",
                  verbose=verbose
                  )
    
    model_list.append(model)

saver(model_list, path2 + "belem_concatenate_2layer_fitted_unitary_validation.model")

  0%|          | 0/10 [00:00<?, ?it/s]

0.009941276679746917 0.011737668506148035
0.008666962151443216 0.011225792761737118
0.009044053589030574 0.010796976935504589
0.009285596226351217 0.011775632845828962
0.009036489368027823 0.012114457908952554
0.009074251413967878 0.010918835496107466
0.010829658228686556 0.013020392235150969
0.010543068225402753 0.014359651501799306
0.010607064547118409 0.01314456507399788
0.010591344792902926 0.012971502690365845


In [8]:
n = 3
d = 2**n
model_list = []
verbose = False

np.random.seed(42)
random.seed(42)
for i in tqdm(range(10)):

    model =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename=path1 + f"belem_concatenate_4layer{i}2",
                  verbose=verbose
                  )
    
    model_list.append(model)

saver(model_list, path2 + "belem_concatenate_4layer_fitted_unitary_validation.model")

  0%|          | 0/10 [00:00<?, ?it/s]

0.007453616269990975 0.009838401763944555
0.008728143911772424 0.010603337867021409
0.008598339416128784 0.010129804349691011
0.008857418954602556 0.01135519434154145
0.00754201163451898 0.010724110865143197
0.0072588467533717385 0.009756072142343293
0.007877432430243997 0.010946398694690874
0.006829735183049644 0.009124473267932882
0.00778115389612004 0.011305719969063703
0.008409434175991918 0.010608780143299272


In [9]:
n = 3
d = 2**n
model_list = []
verbose = False

np.random.seed(42)
random.seed(42)
for i in tqdm(range(10)):

    model =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename=path1 + f"belem_concatenate_6layer{i}2",
                  verbose=verbose
                  )
    
    model_list.append(model)

saver(model_list, path2 + "belem_concatenate_6layer_fitted_unitary_validation.model")

  0%|          | 0/10 [00:00<?, ?it/s]

0.005081020007860076 0.008127009078692087
0.005692399158608174 0.00782466470106713
0.005774244238474972 0.008328456101586576
0.005344265847063093 0.01750381235365891
1.7417980683985543 0.004029658836343039
0.148545857801463 0.049394206593945715
0.004248111488610959 0.00855257767379092
0.0038595174635787538 0.00653416925658373
0.002694977293388596 0.00494719331273565
0.002483875833981289 0.004595967147921732


In [10]:
n = 3
d = 2**n
model_list = []
verbose = False

np.random.seed(42)
random.seed(42)
for i in tqdm(range(10)):

    model =fit_model(channel=KrausMap(d=d, 
                                   rank=d**2),
                  spam=SPAM(init = InitialState(d),
                            povm = CorruptionMatrix(d),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            ),
                  loss_function = ProbabilityMSE(),
                  N_map=500,
                  N_spam=None,
                  num_iter_pretrain=500,
                  num_iter_map=2000,
                  num_iter_spam=500,
                  filename=path1 + f"belem_concatenate_8layer{i}2",
                  verbose=verbose
                  )
    
    model_list.append(model)

saver(model_list, path2 + "belem_concatenate_8layer_fitted_unitary_validation.model")

  0%|          | 0/10 [00:00<?, ?it/s]

0.04323498721763977 0.012232084699607172
0.005423912572729612 0.008037372296650316
0.00565504282408742 0.008086033318584835
0.005338737751978078 0.00871197672861486
0.004859058526872409 0.0076438137671584905
0.005243166074346057 0.008319739245609902
0.005669891069200863 0.008738881241676568
0.005340500925228726 0.008354278483015226
0.005617530752401395 0.00845406726301281
0.004957098319481476 0.007190221398947205
